In [1]:
import os
os.chdir('D:/学习/9444dataset/archive')

In [2]:
# Libraries
import os
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [3]:
from glob import glob
imagePatches = glob('IDC_regular_ps50_idx5/**/*.png', recursive=True)

In [4]:
len(imagePatches)

277524

In [5]:
imagePatches[0]

'IDC_regular_ps50_idx5\\10253\\0\\10253_idx5_x1001_y1001_class0.png'

In [6]:
import fnmatch
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

In [7]:
total_num = 277524
count = 0
y = []
progress_increment = total_num * 0.05  # 5% of total
next_progress = progress_increment

for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)

    count += 1

    if count >= next_progress:
        print(f'Progress: {count/total_num*100:.2f}%')  # Prints the progress as a percentage
        next_progress += progress_increment




Progress: 5.00%
Progress: 10.00%
Progress: 15.00%
Progress: 20.00%
Progress: 25.00%
Progress: 30.00%
Progress: 35.00%
Progress: 40.00%
Progress: 45.00%
Progress: 50.00%
Progress: 55.00%
Progress: 60.00%
Progress: 65.00%
Progress: 70.00%
Progress: 75.00%
Progress: 80.00%
Progress: 85.00%
Progress: 90.00%
Progress: 95.00%


In [8]:
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y

In [9]:
images_df.head()

,images,labels
0,IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...,0
1,IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...,0
2,IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...,0
3,IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...,0
4,IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...,0


In [10]:
images_df.groupby("labels")["labels"].count()

labels
0    198738
1     78786
Name: labels, dtype: int64

In [11]:
#Splitting data into train and val
train, val = train_test_split(images_df, stratify=images_df.labels, test_size=0.2)
len(train), len(val)

(222019, 55505)

In [12]:
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (50,50))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [13]:
## Parameters for model

# Hyper parameters
num_epochs = 10
num_classes = 2
batch_size = 64
learning_rate = 0.002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [14]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [15]:
class SimpleCNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
        # try resnet
        
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        # print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [16]:
model = SimpleCNN().to(device)

In [17]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [18]:
# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/3470], Loss: 0.4762
Epoch [1/10], Step [200/3470], Loss: 0.4637
Epoch [1/10], Step [300/3470], Loss: 0.3023
Epoch [1/10], Step [400/3470], Loss: 0.2685
Epoch [1/10], Step [500/3470], Loss: 0.3490
Epoch [1/10], Step [600/3470], Loss: 0.4811
Epoch [1/10], Step [700/3470], Loss: 0.2725
Epoch [1/10], Step [800/3470], Loss: 0.2610
Epoch [1/10], Step [900/3470], Loss: 0.3298
Epoch [1/10], Step [1000/3470], Loss: 0.3598
Epoch [1/10], Step [1100/3470], Loss: 0.4027
Epoch [1/10], Step [1200/3470], Loss: 0.3930
Epoch [1/10], Step [1300/3470], Loss: 0.2919
Epoch [1/10], Step [1400/3470], Loss: 0.3336
Epoch [1/10], Step [1500/3470], Loss: 0.3963
Epoch [1/10], Step [1600/3470], Loss: 0.3516
Epoch [1/10], Step [1700/3470], Loss: 0.4978
Epoch [1/10], Step [1800/3470], Loss: 0.2064
Epoch [1/10], Step [1900/3470], Loss: 0.2815
Epoch [1/10], Step [2000/3470], Loss: 0.3480
Epoch [1/10], Step [2100/3470], Loss: 0.4088
Epoch [1/10], Step [2200/3470], Loss: 0.4249
Epoch [1/10], Step 

In [19]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
                 
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the test images: 90.42969101882713 %


In [20]:
print(confusion_matrix)

tensor([[36344.,  3404.],
        [ 1908., 13849.]])
